In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [11]:
url = 'https://www.engineeringnz.org/courses-events'
response = requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')

In [12]:
title = []
date_time = []
price = []
CPD_Hours= []

In [13]:
course_data = soup.findAll('div',attrs={'class':'width-1/2@xsmall width-1/3@medium'})

In [14]:
for store in course_data:
    titles  = store.h5.text.replace('\n','').strip()
    title.append(titles)
    
    cost = store.a.p.text.replace('\n','').strip()
    price.append(cost)
    
    datetime = store.a.p.find_all('p',class_='t-color-light-grey u-text-uppercase u-space-xs')
    date_time.append(datetime)
    
    Hours_elems=store.a.p.find_all('p', class_= 'u-text-xs u-text-medium u-text-right')
    Hours =[elem.get_text() for elem in Hours_elems]
    
    CPD_Hours.append(Hours)

In [15]:
title

["Auckland's draft parking strategy",
 'Dunedin quiz night',
 'Timber Design Society | Laboratory Visit',
 'Continuous Commissioning Specialist',
 'A day in the life of a fire safety engineer',
 'Relationship Management in Construction',
 'Unsaturated Soil Mechanics for Slope Failures',
 'Decarbonizing distribution transformers',
 'Women in leadership panel']

In [16]:
price

['Free', 'Free', 'Free', '$1794.00', 'Free', 'Free', 'Free', 'Free', 'Free']

In [17]:
CPD =" ".join([str(item) for item in CPD_Hours])
CPD = " ".join(CPD.split())
CPD = CPD.replace("\\n","")
CPD = CPD.replace("  ","")
CPD_Hours

[[], [], [], [], [], [], [], [], []]

In [18]:
date_time

[[], [], [], [], [], [], [], [], []]